In [1]:
import os

os.environ["BRASA_DATA_PATH"] = "/mnt/d/brasa"

In [2]:
import sys
sys.path.append('..')

from datetime import datetime

import pandas as pd
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pyarrow

import brasa

In [10]:
symbols = brasa.get_symbols("equity", index="IBOV")
len(symbols)

86

In [8]:
rets = brasa.get_returns(symbols, start=datetime(2021, 1, 1))

In [13]:
rets.shape

(899, 86)

In [12]:
rets.dropna(axis=1).to_csv("ibov-stocks.csv")

In [15]:
df = brasa.get_dataset("b3-companies-properties").to_table().to_pandas()
df.head()

,asset_name,company_name,trading_name,cnpj,code_cvm,industry_classification,activity,website,market_indicator,market,...,exchange_segment,has_common,has_preferred,quoted_per_shar_since,round_lot,stock_capital,number_common_shares,number_preferred_shares,total_number_shares,refdate_y
0,AALR,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,ALLIAR,42771949000135,24058,Saúde / Serv.Méd.Hospit..Análises e Diagnóstic...,Prestação de serviços de medicina diagnóstica,http://ri.alliar.com/,18.0,NOVO MERCADO,...,NM,True,True,2016-10-14,100,6.353720e+08,118292816,0,118292816,2023-12-20
1,ABCB,BCO ABC BRASIL S.A.,ABC BRASIL,28195667000106,20958,Financeiro / Intermediários Financeiros / Bancos,O Banco tem por objeto a prática de operações ...,www.abcbrasil.com.br,17.0,BOVESPA NIVEL 2,...,N2,True,True,2007-07-25,100,4.472131e+09,119010510,117925586,236936096,2023-12-20
2,ABEV,AMBEV S.A.,AMBEV S/A,07526557000100,23264,Consumo não Cíclico / Bebidas / Cervejas e Ref...,Fabricação e distribuição de cervejas. refrige...,http://ri.ambev.com.br,1.0,BOLSA,...,BOLSA,True,False,2013-11-01,100,5.817793e+10,15753833284,0,15753833284,2023-12-20
3,AERI,AERIS IND. E COM. DE EQUIP. GERACAO DE ENERGIA...,AERIS,12528708000107,25283,Bens Industriais / Máquinas e Equipamentos / M...,Exploração de negócio de construção e comercia...,,18.0,NOVO MERCADO,...,NM,True,False,2020-10-20,100,9.119750e+08,1242403933,0,1242403933,2023-12-20
4,AESB,AES BRASIL ENERGIA S.A.,AES BRASIL,37663076000107,25640,Utilidade Pública / Energia Elétrica / Energia...,HOLDING DE INSTITUIÇÕES NÃO FINANCEIRAS,http://ri.aesbrasil.com.br,18.0,NOVO MERCADO,...,NM,True,False,2021-03-25,100,2.196958e+09,601927311,0,601927311,2023-12-20


In [17]:
df[["asset_name", "stock_capital", "number_common_shares", "number_preferred_shares", "total_number_shares"]].sort_values("stock_capital", ascending=False).style.format(thousands=",")

,asset_name,stock_capital,number_common_shares,number_preferred_shares,total_number_shares
286,PETR,"205,431,960,490.519989","7,442,454,142","5,602,042,788","13,044,496,930"
36,BBAS,"120,000,000,000.000000","2,865,417,020",0,"2,865,417,020"
199,ITUB,"90,729,000,000.000000","4,958,290,359","4,845,844,989","9,804,135,348"
37,BBDC,"87,100,000,000.000000","5,330,304,681","5,311,865,547","10,642,170,228"
385,VALE,"77,300,000,000.000000","4,539,007,568",12,"4,539,007,580"
198,ITSA,"73,189,000,000.000000","3,549,302,243","6,778,847,188","10,328,149,431"
138,ELET,"70,099,825,620.789993","2,027,011,498","280,088,314","2,307,099,812"
391,VIVT,"63,571,415,865.089996","1,663,556,731",0,"1,663,556,731"
2,ABEV,"58,177,928,601.849998","15,753,833,284",0,"15,753,833,284"
335,SANB,"55,000,000,000.000000","3,818,695,031","3,679,836,020","7,498,531,051"


In [10]:
def get_company_names() -> list:
    tb = brasa.get_dataset("b3-equities-register").scanner(columns=["refdate"]).to_table()
    max_date = pyarrow.compute.max(tb.column("refdate"))
    df = brasa.get_dataset("b3-equities-register")\
        .filter(pc.field("instrument_market") == 10)\
        .filter(pc.field("security_category") == 11)\
        .filter(pc.field("refdate") == max_date)\
        .scanner(columns=["instrument_asset"])\
        .to_table().to_pandas()
    return list(df.instrument_asset.unique())

def get_company_cvm_codes() -> list:
    tb = brasa.get_dataset("b3-company-info-report").scanner(columns=["refdate"]).to_table()
    max_date = pyarrow.compute.max(tb.column("refdate"))
    df = brasa.get_dataset("b3-company-info-report")\
        .filter(pc.field("refdate") == max_date)\
        .filter(pc.field("codeCVM") != "0")\
        .scanner(columns=["codeCVM"])\
        .to_table().to_pandas()
    return list(df.codeCVM.unique())

def get_company_trading_names() -> list:
    tb = brasa.get_dataset("b3-company-details").scanner(columns=["refdate"]).to_table()
    max_date = pyarrow.compute.max(tb.column("refdate"))
    df = brasa.get_dataset("b3-company-details")\
        .filter(pc.field("refdate") == max_date)\
        .scanner(columns=["tradingName"])\
        .to_table().to_pandas()
    return list(df.tradingName.unique())

In [11]:
get_company_trading_names()

['BRASIL',
 'ALPARGATAS',
 'SONDOTECNICA',
 'WLM IND COM',
 'BANESE',
 'TEKA',
 'TEKNO',
 'TELEBRAS',
 'OI',
 'CRISTAL',
 'BANESTES',
 'UNIPAR',
 'BANPARA',
 'VULCABRAS',
 'IOCHP-MAXION',
 'AZEVEDO',
 'WETZEL S/A',
 'BANRISUL',
 'BOMBRIL',
 'NORD BRASIL',
 'RECRUSUL',
 'KLABIN S/A',
 'SANSUY',
 'ALIPERTI',
 'MERC INVEST',
 'MERCANTIL',
 'JOSAPAR',
 'HAGA S/A',
 'RIOSULENSE',
 'POLPAR',
 'PLASCAR PART',
 'MINUPAR',
 'PORTOBELLO',
 'SAO CARLOS',
 'ALFA INVEST',
 'CEMEPE',
 'SUZANO S.A.',
 'RANDON PART',
 'BRB BANCO',
 'COPEL',
 'USIMINAS',
 'WHIRLPOOL',
 'SABESP',
 'CEB',
 'CYRELA REALT',
 'COELBA',
 'ENERGISA MT',
 'SCHULZ',
 'CIMS',
 'P.ACUCAR-CBD',
 'COELCE',
 'LITEL',
 'BARDELLA',
 'ENERGISA',
 'ALL NORTE',
 'FER C ATLANT',
 'ATOMPAR',
 'EMBPAR S/A',
 'NEOENERGIA',
 'BAUMER',
 'COMGAS',
 'EXCELSIOR',
 'ELETROPAR',
 'GAFISA',
 '524 PARTICIP',
 'BRF SA',
 'ROSSI RESID',
 'SUL 116 PART',
 'SUDESTE S/A',
 'EQTLMARANHAO',
 'CEG',
 'UPTICK',
 'DEXXOS PAR',
 'PORTO SEGURO',
 'YBYRA S/A',
 '

In [9]:
from brasa import download_marketdata, process_marketdata

download_marketdata("b3-cash-dividends", tradingName=["ALPARGATAS"])
# process_marketdata("b3-company-details")

In [4]:
company_names = get_company_names()
company_names[:10]

['LOGG',
 'TTEN',
 'BFFT',
 'ALPK',
 'SULG',
 'JFEN',
 'MEAL',
 'SHOW',
 'PMAM',
 'EESG']

In [23]:
company_cvm_codes = get_company_cvm_codes()
company_cvm_codes[:10]

['24058',
 '20958',
 '23264',
 '25283',
 '25640',
 '26174',
 '22179',
 '909705',
 '183',
 '20036']

In [5]:
len(company_names), len(company_cvm_codes)

(530, 426)

In [24]:
from brasa.engine import retrieve_template, CacheMetadata, CacheManager
import json
import gzip

template_name = "b3-indexes-theoretical-portfolio-with-sectors"
template = retrieve_template(template_name)
meta = CacheMetadata(template.id)
meta.extra_key = template.downloader.extra_key
cache = CacheManager()
meta.download_args = dict(index="IBOV")
cache.download_marketdata(meta)

with gzip.open(os.path.join("D:\\brasa", meta.downloaded_files[0])) as f:
        s = f.read()
        obj = json.loads(s)
obj

{'page': {'pageNumber': 1,
  'pageSize': 9999,
  'totalRecords': 86,
  'totalPages': 1},
 'header': {'date': '22/12/23',
  'text': 'Quantidade Teórica Total',
  'part': '100,000',
  'partAcum': '100,000',
  'textReductor': 'Redutor',
  'reductor': '17.205.758,63627720',
  'theoricalQty': '96.571.029.272'},
 'results': [{'segment': 'Bens Indls / Máqs e Equips',
   'cod': 'WEGE3',
   'asset': 'WEG',
   'type': 'ON  EJ  NM',
   'part': '2,371',
   'partAcum': '2,371',
   'theoricalQty': '1.481.593.024'},
  {'segment': 'Bens Indls / Mat Transporte',
   'cod': 'EMBR3',
   'asset': 'EMBRAER',
   'type': 'ON      NM',
   'part': '0,731',
   'partAcum': '0,731',
   'theoricalQty': '734.632.705'},
  {'segment': 'Bens Indls/Transporte',
   'cod': 'AZUL4',
   'asset': 'AZUL',
   'type': 'PN      N2',
   'part': '0,236',
   'partAcum': '2,165',
   'theoricalQty': '327.593.725'},
  {'segment': 'Bens Indls/Transporte',
   'cod': 'CCRO3',
   'asset': 'CCR SA',
   'type': 'ON      NM',
   'part': '0,6

In [26]:
pd.DataFrame(obj["results"])

,segment,cod,asset,type,part,partAcum,theoricalQty
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON EJ NM,"2,371","2,371",1.481.593.024
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,"0,731","0,731",734.632.705
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,"0,236","2,165",327.593.725
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,"0,628","2,165",995.335.937
4,Bens Indls/Transporte,GOLL4,GOL,PN N2,"0,079","2,165",198.184.909
...,...,...,...,...,...,...,...
81,Utilidade Públ / Energ Elétrica,ENGI11,ENERGISA,UNT N2,"0,657","10,089",290.435.910
82,Utilidade Públ / Energ Elétrica,ENEV3,ENEVA,ON NM,"0,939","10,089",1.578.931.315
83,Utilidade Públ / Energ Elétrica,EGIE3,ENGIE BRASIL,ON EJ NM,"0,499","10,089",255.230.961
84,Utilidade Públ / Energ Elétrica,EQTL3,EQUATORIAL,ON NM,"1,700","10,089",1.095.587.251


In [13]:
df_companies = brasa.get_dataset("b3-company-info-report").to_table().to_pandas()
df_companies.loc[df_companies["codeCVM"] == "25992", "code"]

In [6]:
brasa.get_dataset("b3-company-info-cash_dividends").to_table().to_pandas()

,assetIssued,paymentDate,rate,relatedTo,approvedOn,isinCode,label,lastDatePrior,remarks,refdate
0,BRAALRACNOR6,2020-12-18,0.087298,ANUAL/2019,2020-04-27,BRAALRACNOR6,DIVIDENDO,2020-04-27,,2023-12-20
1,BRABCBACNPR4,2023-08-16,0.836000,1º Semestre/2023,2023-06-26,BRABCBACNPR4,JRS CAP PROPRIO,2023-06-30,,2023-12-20
2,BRABCBACNPR4,2023-01-13,0.370000,4º Trimestre/2022,2022-12-26,BRABCBACNPR4,JRS CAP PROPRIO,2022-12-29,,2023-12-20
3,BRABEVACNOR1,2023-12-28,0.730200,4º Trimestre/2023,2023-12-12,BRABEVACNOR1,JRS CAP PROPRIO,2023-12-19,,2023-12-20
4,BRABEVACNOR1,2021-12-30,0.133400,Anual/2021,2021-12-09,BRABEVACNOR1,DIVIDENDO,2021-12-17,,2023-12-20
...,...,...,...,...,...,...,...,...,...,...
1674,BRWLMMACNPR3,2023-06-27,0.169400,ANUAL/2022,2023-03-20,BRWLMMACNPR3,DIVIDENDO,2023-04-28,,2023-12-20
1675,BRWLMMACNPR3,2023-06-27,0.269500,COMPLEMENTAR/2022,2023-03-20,BRWLMMACNPR3,DIVIDENDO,2023-04-28,,2023-12-20
1676,BRWLMMACNOR6,2023-02-27,0.260423,4º Trimestre/2022,2022-12-29,BRWLMMACNOR6,JRS CAP PROPRIO,2023-01-06,,2023-12-20
1677,BRWLMMACNPR3,2023-02-27,0.286466,4º Trimestre/2022,2022-12-29,BRWLMMACNPR3,JRS CAP PROPRIO,2023-01-06,,2023-12-20


In [4]:
brasa.get_dataset("b3-company-details").to_table().to_pandas()

,issuingCompany,companyName,tradingName,cnpj,industryClassification,industryClassificationEng,activity,website,hasQuotation,status,...,institutionPreferred,code,codeCVM,lastDate,hasEmissions,hasBDR,typeBDR,describleCategoryBVMF,isin,refdate
0,BBAS,BCO BRASIL S.A.,BRASIL,00000000000191,Financeiro / Intermediários Financeiros / Bancos,None,Banco Múltiplo,www.bb.com.br,None,A,...,BRASIL,BBAS11,1023,20/12/2023 23:15:03,False,False,None,None,BRBBASA04OR8,2023-12-21
1,BBAS,BCO BRASIL S.A.,BRASIL,00000000000191,Financeiro / Intermediários Financeiros / Bancos,None,Banco Múltiplo,www.bb.com.br,None,A,...,BRASIL,BBAS12,1023,20/12/2023 23:15:03,False,False,None,None,BRBBASA05OR5,2023-12-21
2,BBAS,BCO BRASIL S.A.,BRASIL,00000000000191,Financeiro / Intermediários Financeiros / Bancos,None,Banco Múltiplo,www.bb.com.br,None,A,...,BRASIL,BBAS3,1023,20/12/2023 23:15:03,False,False,None,None,BRBBASACNOR3,2023-12-21
3,ALPA,ALPARGATAS S.A.,ALPARGATAS,61079117000105,Consumo Cíclico / Tecidos. Vestuário e Calçado...,None,Indústria e comércio de calçados e artigos esp...,www.alpargatas.com.br,None,A,...,ITAU CORRETORA ACOES,ALPA3,10456,20/12/2023 23:09:36,False,False,None,None,BRALPAACNOR0,2023-12-21
4,ALPA,ALPARGATAS S.A.,ALPARGATAS,61079117000105,Consumo Cíclico / Tecidos. Vestuário e Calçado...,None,Indústria e comércio de calçados e artigos esp...,www.alpargatas.com.br,None,A,...,ITAU CORRETORA ACOES,ALPA4,10456,20/12/2023 23:09:36,False,False,None,None,BRALPAACNPR7,2023-12-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,PTNT,PETTENATI S.A. INDUSTRIA TEXTIL,PETTENATI,88613658000110,Consumo Cíclico / Tecidos. Vestuário e Calçado...,None,None,www.pettenati.com.br,None,A,...,ITAU CORRETORA ACOES,PTNT4,9539,20/12/2023 23:48:49,False,False,None,None,BRPTNTACNPR3,2023-12-21
608,RPAD,ALFA HOLDINGS S.A.,ALFA HOLDING,17167396000169,Financeiro / Intermediários Financeiros / Bancos,None,Emp. Adm. Part. - Intermediação Financeira,www.alfanet.com.br,None,A,...,SANTANDER,RPAD3,9954,20/12/2023 23:09:08,False,False,None,None,BRRPADACNOR1,2023-12-21
609,RPAD,ALFA HOLDINGS S.A.,ALFA HOLDING,17167396000169,Financeiro / Intermediários Financeiros / Bancos,None,Emp. Adm. Part. - Intermediação Financeira,www.alfanet.com.br,None,A,...,SANTANDER,RPAD5,9954,20/12/2023 23:09:08,False,False,None,None,BRRPADACNPA4,2023-12-21
610,RPAD,ALFA HOLDINGS S.A.,ALFA HOLDING,17167396000169,Financeiro / Intermediários Financeiros / Bancos,None,Emp. Adm. Part. - Intermediação Financeira,www.alfanet.com.br,None,A,...,SANTANDER,RPAD6,9954,20/12/2023 23:09:08,False,False,None,None,BRRPADACNPB2,2023-12-21


In [15]:
import base64

base64.b64decode("eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjIwLCJ0cmFkaW5nTmFtZSI6IkJSQVNJTCJ9")

b'{"language":"pt-br","pageNumber":1,"pageSize":20,"tradingName":"BRASIL"}'